# Python setup

In [1]:
import subprocess 
import time
import glob
import os
from pathlib import Path
import shutil
import pandas as pd
import tempfile
from PIL import Image
import resource
from ast import literal_eval

def run(obj,print_output=False):
    start = time.perf_counter()
    result = subprocess.run(obj["args"],capture_output=True,text=True)    
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        result.check_returncode()
    else:
        if print_output:
            print(result.stdout)
        return { "delta": time.perf_counter() - start, "label": obj["label"],"output": result.stdout,"error": result.stderr, "return_code": result.returncode }
    
rid="linux-x64"
#converter_exe_rel="../../bin/release/net6.0/linux-x64/GraphLogicA"
converter_exe_rel=f"../../bin/release/net6.0/{rid}/GraphLogicA"
converter_exe = Path(converter_exe_rel).absolute().as_posix()
graphlogica_exe = converter_exe
minimizer_exe = shutil.which("ltsconvert")
voxlogica_exe = f"../../VoxLogicA_1.0-experimental_{rid}/VoxLogicA"
output="output"
#shutil.rmtree(output,ignore_errors=True)
os.makedirs(output,exist_ok=True)
images = glob.glob("test-images/*.png")
    
def mk_df(results,delta_label):
    return pd.DataFrame(results).set_index("label").rename(columns={"delta": delta_label}).drop(columns=["output","error","return_code"])
#!(cd ../.. && dotnet build -c release -r $rid)
resource.setrlimit(resource.RLIMIT_STACK, (resource.RLIM_INFINITY, resource.RLIM_INFINITY))

# Convert images

In [2]:
def converter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix()], "label": label }
    #return (run(label,args))

converter_df = mk_df([ run(converter(image)) for image in images ],"conversionAndWrite")


# Minimize

In [3]:
def minimizer(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    return { "args": [minimizer_exe,"-ebranching-bisim",d_path.as_posix(),m_path.as_posix()], "label": label }

minimizer_df = mk_df([ run(minimizer(image)) for image in images ],"minimization")


# Convert without writing the file 

In [4]:
def fakeConverter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".fake.aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix(),"--fakeconversion"], "label": label }
    #return (run(label,args))

fakeConverter_df = mk_df([ run(fakeConverter(image)) for image in images ],"conversion")


# Model checking on images using VoxLogicA

In [5]:
def mazeSpecification(path):
    return f'''
    load img = "{path}"
    let colour(img,r,g,b) = (red(img) =. r) & (green(img) =. g) & (blue(img) =. b)
    let w = colour(img,255,255,255)
    let b = colour(img,0,0,255)
    let g = colour(img,0,255,0)
    let form1 = through(N b,w) & through(N g,w)
    let form2 = through(N (w & !through(N g,w)),b) & through(N g,w)
    let form3 = through(N form1,b)
    save "output/{path}/form1.png" form1
    save "output/{path}/form2.png" form2
    save "output/{path}/form3.png" form3
    '''

def modelChecker(image):
    path = Path(image)
    spec = mazeSpecification(image)
    fname = tempfile.NamedTemporaryFile().name
    f = open(fname, "w")
    f.write(spec)
    f.close()
    return { "args": [voxlogica_exe,fname], "label": path.name }

modelChecker_df = mk_df([ run(modelChecker(image)) for image in images ],"modelCheckingFull")


# Model Checking on the minimal graph using GraphLogicA

In [6]:
def graphSpecification():
    gql = f'''
    load graph = "mazeMin.json"
    let w = ap("cFFFFFF")
    let b = ap("c0000FF")
    let g = ap("c00FF00")
    let form1 = through(N b,w) & through(N g,w)
    let form2 = through(N (w & !through(N g,w)),b) & through(N g,w)
    let form3 = through(N form1,b)
    save "output/form1.json" form1
    save "output/form2.json" form2
    save "output/form3.json" form3
    '''
    fname = tempfile.NamedTemporaryFile().name
    f = open(fname,"w")
    f.write(gql)
    f.close()
    return run({"args" : [graphlogica_exe,fname], "label": "modelCheckingMin"},print_output=True)

graph_delta = graphSpecification()["delta"]



[        11ms] [info] GraphLogicA 0.4.1
[       206ms] [info] Parsing input...
[       227ms] [info] Preparing computation...
[       233ms] [info] Importing file "/home/vincenzo/data/local/repos/GraphLogicA/src/bin/release/net6.0/linux-x64/stdlib.imgql"
[       285ms] [info] Starting computation...
[       286ms] [info] Running 19 tasks
save to 'save to 'save to '"/home/vincenzo/data/local/repos/GraphLogicA/src/test/min/output/form1.json"': "/home/vincenzo/data/local/repos/GraphLogicA/src/test/min/output/form2.json"': "/home/vincenzo/data/local/repos/GraphLogicA/src/test/min/output/form3.json"': [|false; false; false; false; false; false; true|][|false; false; false; false; false; true; false|][|false; false; false; false; false; false; false|]


[       309ms] [info] ... done.



In [7]:
graph_true_delta = 0.02 # gotten by hand looking at the output

In [8]:
# Read automata statistics

def autSize(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    first_line = ""
    first_line_min = ""
    with open(d_path,"r") as file:
        first_line = file.readline().lstrip("des ")
    
    with open(m_path,"r") as file:
        first_line_min = file.readline().lstrip("des ")

    t = literal_eval(first_line)
    tmin = literal_eval(first_line_min)

    return { "states": t[2], "transitions": t[1], "statesMin": tmin[2] , "transitionsMin": tmin[1], "label": label }

autSize_df = pd.DataFrame([ autSize(image) for image in images]).set_index("label")

    

# Gather image sizes and produce the final table

In [9]:
def size(imgpath):    
    path = Path(imgpath)
    img = Image.open(imgpath)
    return { "pixels": img.width * img.height, "label": path.name}

size_df = pd.DataFrame([ size(image) for image in images]).set_index("label")

In [10]:
df = size_df.join(autSize_df).join(fakeConverter_df).join(converter_df).join(minimizer_df).join(modelChecker_df)
df["modelCheckingMin"] = graph_delta
df["modelCheckingMinLimit"] = graph_true_delta
df["idealTime"] = df["conversion"] + df["modelCheckingMinLimit"]
df["idealSpeedup"] = df["modelCheckingFull"] / df["idealTime"]
data = df.sort_values(by='pixels')

# Export data

In [11]:
df.to_latex("table.tex")